In [11]:
from collections import namedtuple

In [12]:
Files = namedtuple('filelist',['categorical','date','numeric'])
Data = namedtuple('data',['categorical','date','numeric'])
train_files = Files('./Data/train_categorical.csv.gz',
                './Data/train_date.csv.gz',
                './Data/train_numeric.csv.gz')
test_files = Files('./Data/test_categorical.csv.gz',
                './Data/test_date.csv.gz',
                './Data/test_numeric.csv.gz')

In [13]:
# train  
train = Data(sc.textFile(train_files.categorical).cache(),
             sc.textFile(train_files.date).cache(),
             sc.textFile(train_files.numeric).cache())

# test 
test = Data(sc.textFile(test_files.categorical).cache(),
             sc.textFile(test_files.date).cache(),
             sc.textFile(test_files.numeric).cache())

In [ ]:
remove_header = lambda x: x.split(',')[0]!='Id'
key_split = lambda x: (x.split(',')[0], x.split(',')[1:])
convert_numeric = lambda (k,v): (k, float(v))
train_filtered = Data(
    train.categorical.filter(remove_header).map(key_split).repartition(100),
    train.date.filter(remove_header).map(key_split).repartition(100),
    train.numeric.filter(remove_header).map(key_split).map(convert_numeric).repartition(100)
)
test_filtered = Data(
    test.categorical.filter(remove_header).map(key_split).repartition(100),
    test.date.filter(remove_header).map(key_split).repartition(100),
    test.numeric.filter(remove_header).map(key_split).map(convert_numeric).repartition(100)
)

In [ ]:
train_filtered.categorical.first()